# Serve an Agent with A2A

Once you've built an agent, a common question might be: "how do I let other developers/applications access it?". Enter the [A2A](https://github.com/google-a2a/A2A) protocol by Google! A2A is "An open protocol enabling communication and interoperability between opaque agentic applications". Any-Agent provides support for serving an agent over A2A, as simple as calling `agent.serve()`. In this tutorial, we'll build and serve an agent using any-agent, and show how you can serve and interact with it via the A2A protocol.

<img src="/images/serve_a2a.png" alt="" style="max-width: 500px; height: auto;">

This tutorial assumes basic familiarity with any-agent: if you haven't used any-agent before you may also find the [Creating your first agent](./../your_first_agent) cookbook to be useful.

Note: because this tutorial relies upon advanced stdio/stderr communication using the MCP Server, it cannot be run on Google Colab.


## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent[a2a]'

import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
from getpass import getpass

# This notebook communicates with OpenAI GPT models using the OpenAI API.
if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")

OPENAI_API_KEY found in environment.


## Configure and run the server

Let's give our agent the very simple capability to access the current time through a Model Context Protocol (MCP) server. For this demo, we'll use the async method `agent.serve_async` so that we can easily run both the server and client from inside the notebook. 

In [ ]:
from any_agent import AgentConfig, AnyAgent
from any_agent.config import MCPStdio, ServingConfig, TracingConfig

# This MCP Tool relies upon uvx https://docs.astral.sh/uv/getting-started/installation/
time_tool = MCPStdio(
    command="uvx",
    args=["mcp-server-time", "--local-timezone=America/New_York"],
    tools=[
        "get_current_time",
    ],
)

time = AnyAgent.create(
    "tinyagent",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(
        model_id="gpt-4.1-mini",
        description="I'm an agent to help with getting the time",
        tools=[time_tool],
    ),
    tracing=TracingConfig(console=False),
)
(time_task, time_server) = await time.serve_async(ServingConfig(port=5000))

## Call the agent using A2AClient

Now that the agent is listening on http://localhost:5000, we can communicate with it from any other application that supports A2A. For this tutorial we'll use the a2a python SDK, but any client that implements the [A2A Protocol](https://github.com/google-a2a/A2A) would also work.

### A2A Agent Card

Before giving the agent a task, we first retrieve the agent's model card, which is a description of the agents capabilities. This helps the client understand what the agent can do, and what A2A features are available.

In [4]:
import httpx
from a2a.client import A2ACardResolver, A2AClient
from a2a.types import AgentCard

# Create the httpx client
httpx_client = httpx.AsyncClient()

agent_card: AgentCard = await A2ACardResolver(
    httpx_client,
    base_url="http://localhost:5000",
).get_agent_card(http_kwargs=None)
print(agent_card.model_dump_json(indent=2))

client = A2AClient(httpx_client=httpx_client, agent_card=agent_card)

{
  "capabilities": {
    "pushNotifications": false,
    "stateTransitionHistory": false,
    "streaming": false
  },
  "defaultInputModes": [
    "text"
  ],
  "defaultOutputModes": [
    "text"
  ],
  "description": "I'm an agent to help with getting the time",
  "documentationUrl": null,
  "name": "any_agent",
  "provider": null,
  "security": null,
  "securitySchemes": null,
  "skills": [
    {
      "description": "Get current time in a specific timezones",
      "examples": null,
      "id": "any_agent-get_current_time",
      "inputModes": null,
      "name": "get_current_time",
      "outputModes": null,
      "tags": []
    }
  ],
  "supportsAuthenticatedExtendedCard": null,
  "url": "http://localhost:5000/",
  "version": "0.1.0"
}


### Make a request of the agent

Now that we've connected to the agent with the A2AClient, we're ready to make a request of the agent! 

In [6]:
from uuid import uuid4

from a2a.types import MessageSendParams, SendMessageRequest

send_message_payload = {
    "message": {
        "role": "user",
        "parts": [{"kind": "text", "text": "What time is it?"}],
        "messageId": uuid4().hex,
    },
}
request = SendMessageRequest(params=MessageSendParams(**send_message_payload))
response = await client.send_message(request)
# Close the httpx client when done
await httpx_client.aclose()

In [7]:
print(response.model_dump_json(indent=2))

{
  "id": "8c8d295b-1ef4-4ce4-9b9a-7edf65da27f1",
  "jsonrpc": "2.0",
  "result": {
    "contextId": null,
    "kind": "message",
    "messageId": "05c929d8-f009-4a59-a0a0-a1035a3e2099",
    "metadata": null,
    "parts": [
      {
        "kind": "text",
        "metadata": null,
        "text": "The current time in New York (Eastern Time) is 6:20 AM on May 28, 2025. If you want the time for another timezone, please tell me which one."
      }
    ],
    "referenceTaskIds": null,
    "role": "agent",
    "taskId": null
  }
}


## Cleanup

In order to shut down the A2A server, we can set the `should_exit` property of the server, which will cause the server to shutdown and the asyncio task to complete.

In [8]:
import asyncio

time_server.should_exit = True
while True:
    if time_task.done():
        print("Agent server task has completed!")
        break
    else:
        print("Waiting for server to exit")
        await asyncio.sleep(1)

Waiting for server to exit
Agent server task has completed!
